In [32]:
# import gradio as gr
# import requests
# from langchain.llms import Ollama

import gradio as gr
import requests
from langchain.llms import Ollama
import time
from functools import lru_cache
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# TMDB API 키
TMDB_KEY = "97f5ea5f12fa2ebcb7d31beb4ad328b0"

# Ollama 모델 사용할 때는 API 키 필요 없습니다
# Ollama 모델 초기화
model = Ollama(model="gemma2", base_url="http://localhost:11434")

# ========== 성능 최적화 1: 캐싱 시스템 ==========
# 수정됨
@lru_cache(maxsize=200)  # 최대 200개 결과 캐싱
def cached_movie_search(title):
    """영화 검색 결과 캐싱"""
    return flim(title)

@lru_cache(maxsize=200)
def cached_tv_search(title):
    """TV 프로그램 검색 결과 캐싱"""
    return TV(title)

# 1. TMDB API 요청 함수
# "TMDB 정보"를 받아옴
@lru_cache(maxsize=200)
def flim(title):
    
    url = "https://api.themoviedb.org/3/search/movie"
    # 주소 뒤 파라미터
    params = {
        'api_key': TMDB_KEY,
        'query': title,
        'language': 'ko-KR'
    }
    
    start_time = time.time()

    try:
        #url 응답받기
        response = requests.get(url, params=params, timeout=5)
        
        # API 응답 상태 확인
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        
        # 수정됨 # 응답시간 체크
        api_time = time.time() - start_time
        logger.info(f"TMDB API 응답 시간: {api_time:.2f}초")
        
        # "json" data로 응답
        data = response.json()
                       # 에러를 방지해줍니다.
        results = data.get('results', [])
        print("결과:", results)
        print("결과2:", results[0])
        # 결과가 없는 경우 처리
        if not results:
            return f"'{title}'에 대한 영화를 찾을 수 없습니다. 다른 제목을 입력해 보세요."

        # 첫 번째 영화 "정보 추출"
        # "첫번째"만 가져옴 [0]
        movie = results[0]
        movie_info = f"""🎬 제목: {movie.get('title')}
        📅 개봉일: {movie.get('release_date')}
        ⭐ 평점: {movie.get('vote_average')}/10
        📖 줄거리: {movie.get('overview')}
        """

        # 포스터만 보여주면 됨
        poster_path = movie.get('poster_path')
                                                   # 수정됨
        poster_url = f"https://image.tmdb.org/t/p/w300{poster_path}"
        # "https://image.tmdb.org/t/p/w500{poster_path}" : 포스터 경로 
        return movie_info, poster_url

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {str(e)}"
    except IndexError:
        return f"영화 정보를 처리하는 중 오류 발생: 검색 결과가 없습니다."



# ========== 1. 고속 처리용 (가장 빠름) ==========
def get_fast_prompt(movie_info, message):
#     """응답 속도 최우선 - 1-3초 내 응답"""
#     return f""" 당신은 친근하고 전문적인 영화 전문가 챗봇입니다. 
#     다음 영화 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.

# 영화 정보 : {movie_info}

# 사용자 질문 : {message}
# 요구사항: 사용자들이 ~설명해주세요 혹은 ~알려주세요라고 해도 영화정보를 알려주시면 됩니다.
# 위 정보를 바탕으로 영화에 대해 친근하고 상세하게 설명해주세요."""

 """상세하면서도 안정적인 프롬프트 - 답변 생성 보장"""
 return f"""영화 전문가로서 이 영화를 소개해주세요:

{movie_info}

다음 형식으로 답변해주세요:
- 한 줄 요약
- 볼만한 이유 2가지
- 간단한 추천 멘트

총 5줄 이내로 작성해주세요."""

# 2. 챗봇 함수
def flim_chatbot(message):
    
    
#     # 1. TMDB로부터 영화 정보 받아오기
#     movie_info = flim(message)
    
#     # 영화 정보를 찾지 못한 경우 그대로 반환
#     if "찾을 수 없습니다" in movie_info or "오류 발생" in movie_info:
#         return movie_info

#     # 튜플 언패킹(Tuple Unpacking)
#     # 분리
#     movie_info, poster_url = movie_info



#     # 2. AI 모델에 전달할 "프롬프트 구성과 이해"
#     prompt = f"""당신은 친근하고 전문적인 영화 전문가 챗봇입니다. 
# 다음 영화 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.

# 영화 정보: {movie_info} 
#
# 사용자 질문: {message}

# 사용자들이 ~설명해주세요 혹은 ~알려주세요라고 해도 영화정보를 알려주시면 됩니다.
# 위 정보를 바탕으로 영화에 대해 친근하고 상세하게 설명해주세요."""



#     try:
#         # Ollama 모델 호출 
#         # Ollama 모델에 전달하여 응답을 생성합니다. (invoke)
#         response = model.invoke(prompt)
#         return poster_url, response
    
#     except Exception as e:
#         return f"챗봇 응답 생성 중 오류 발생: {str(e)}"
    
            # 수정됨
            movie_info = flim(message)
            
            # if "찾을 수 없습니다" in movie_info or "오류 발생" in movie_info:
            #     return None, movie_info
            
            
            movie_info, poster_url = movie_info
            
            # 고속 프롬프트 사용
            prompt = get_fast_prompt(movie_info, message)
            start_time = time.time()
            
            try:
                response = model.invoke(prompt)
                # return poster_url, response
                # 종료 시간 기록
                end_time = time.time()
                elapsed_time = end_time - start_time
                return poster_url, f"{response}\n\n소요 시간: {elapsed_time:.2f}초"
            except Exception as e:
                # return poster_url, f"빠른 응답 생성 중 오류가 발생했습니다: {str(e)}"
                end_time = time.time()
                elapsed_time = end_time - start_time
                return f"챗봇 응답 생성 중 오류 발생: {str(e)}\n\n소요 시간: {elapsed_time:.2f}초"

# Gradio 인터페이스 생성
iface = gr.Interface(
    # 함수만 받습니다
    fn=flim_chatbot,
    # example
    inputs=gr.Textbox(lines=10, placeholder="영화 제목을 입력하세요 (예: 기생충)"),
    outputs=[gr.Image(label="포스터", show_label=True),  
             gr.Textbox(label="상세설명", lines=20)
                   
    ],
    title="🎬 영화 정보 챗봇",
    description="영화 제목을 입력하면 상세한 정보와 함께 AI가 영화에 대해 설명해드립니다!",
    # examples=[
    #     ["기생충"],
    #     ["어벤져스가 뭐야?"],
    #     ["타이타닉에 대해 알려줘"],
    #     ["인터스텔라는 재미있어?"],
    #     ["기생충이라는 영화 있어?"],
    #     ["탑건 설명해줘"]
    # ]
)




# TMDB API 요청 함수
def TV(title):
    url = "https://api.themoviedb.org/3/search/tv"
    # 주소 뒤 파라미터
    params = {
        'api_key': TMDB_KEY,
        'query': title,
        'language': 'ko-KR'
    }

    try:
        #url 응답받기
        response = requests.get(url, params=params)
        
        # API 응답 상태 확인
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        
        # json data로 응답
        data = response.json()
                       # 에러를 방지해줍니다.
        results = data.get('results', [])
        print("결과:", results)
        print("결과2:", results[0])
        # 결과가 없는 경우 처리
        if not results:
            return f"'{title}'에 대한 프로그램을 찾을 수 없습니다. 다른 제목을 입력해 보세요."

        # 첫 번째 영화 "정보 추출"
        # "첫번째"만 가져옴 [0]
        show = results[0]
        show_info = f"""🎬 제목: {show.get('title')}
        📅 개봉일: {show.get('release_date')}
        ⭐ 평점: {show.get('vote_average')}/10
        📖 줄거리: {show.get('overview')}
        """
        
        
        # 포스터만 보여주면 됨
        poster_path = show.get('poster_path')
        poster_url = f"https://image.tmdb.org/t/p/w500{poster_path}"
        # "https://image.tmdb.org/t/p/w500{poster_path}" : 포스터 경로 
        return show_info, poster_url

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {str(e)}"
    except IndexError:
        return f"영화 정보를 처리하는 중 오류 발생: 검색 결과가 없습니다."

# 챗봇 함수
def show_chatbot(message):
    # 1. TMDB로부터 드라마 정보 받아오기
    show_info = TV(message)
    
    # 드라마 정보를 찾지 못한 경우 그대로 반환
    if "찾을 수 없습니다" in show_info or "오류 발생" in show_info:
        return show_info

    # 튜플 언패킹(Tuple Unpacking)
    # 분리
    show_info, poster_url = show_info

    # 2. AI 모델에 전달할 "프롬프트 구성과 이해"
    prompt = f"""당신은 친근하고 전문적인 TV프로그램 전문가 챗봇입니다. 
다음 TV프로그램 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.


TV프로그램 정보: {show_info} 

사용자 질문: {message}

사용자들이 ~설명해주세요 혹은 ~알려주세요라고 해도 영화정보를 알려주시면 됩니다.
위 정보를 바탕으로 TV프로그램에 대해 친근하고 상세하게 설명해주세요."""

    try:
        # Ollama 모델 호출 
        # Ollama 모델에 전달하여 응답을 생성합니다. (invoke)
        response = model.invoke(prompt)
        return poster_url, response
    
    except Exception as e:
        return f"챗봇 응답 생성 중 오류 발생: {str(e)}"

# Gradio 인터페이스 생성
iface2 = gr.Interface(
    # 함수만 받습니다
    fn=show_chatbot,
    # example
    inputs=gr.Textbox(lines=10, placeholder="TV프로그램 제목을 입력하세요 (예: 고든램지의 헬스키친)"),
    outputs=[gr.Image(label="포스터", show_label=True),  
             gr.Textbox(label="상세설명", lines=20)
                   
    ],
    title="🎬 TV프로그램 정보 챗봇",
    description="TV프로그램 제목을 입력하면 상세한 정보와 함께 AI가 TV프로그램에 대해 설명해드립니다!",
    # examples=[
    #     ["기생충"],
    #     ["어벤져스가 뭐야?"],
    #     ["타이타닉에 대해 알려줘"],
    #     ["인터스텔라는 재미있어?"],
    #     ["기생충이라는 영화 있어?"],
    #     ["탑건 설명해줘"]
    # ]
)


app = gr.TabbedInterface([iface, iface2], ["영화", "드라마"])

# 캐싱 효과 테스트 함수
def test_caching_performance(title, num_calls=5):
    """캐싱과 비캐싱 함수의 성능 비교"""
    logger.info(f"\n=== '{title}'에 대한 성능 테스트 시작 (호출 횟수: {num_calls}) ===")
    
    # 캐싱 적용 함수 테스트
    logger.info("\n캐싱 적용 함수 (@lru_cache 사용)")
    cached_times = []
    for i in range(num_calls):
        start_time = time.perf_counter()
        result = cached_movie_search(title)
        elapsed_time = time.perf_counter() - start_time
        cached_times.append(elapsed_time)
        logger.info(f"호출 {i+1}: {elapsed_time:.4f}초")
    
    avg_cached_time = sum(cached_times) / len(cached_times)
    logger.info(f"캐싱 적용 평균 실행 시간: {avg_cached_time:.4f}초")
    
    # 캐싱 미적용 함수 테스트
    logger.info("\n캐싱 미적용 함수")
    non_cached_times = []
    for i in range(num_calls):
        start_time = time.perf_counter()
        result = flim(title)
        elapsed_time = time.perf_counter() - start_time
        non_cached_times.append(elapsed_time)
        logger.info(f"호출 {i+1}: {elapsed_time:.4f}초")
    
    avg_non_cached_time = sum(non_cached_times) / len(non_cached_times)
    logger.info(f"캐싱 미적용 평균 실행 시간: {avg_non_cached_time:.4f}초")
    
    # 성능 비교
    speedup = avg_non_cached_time / avg_cached_time if avg_cached_time > 0 else float('inf')
    logger.info(f"\n캐싱으로 인한 속도 향상 배율: {speedup:.2f}x")
    logger.info(f"캐싱 시간 단축량: {(avg_non_cached_time - avg_cached_time):.4f}초")
    
    return cached_times, non_cached_times

# 테스트 실행
if __name__ == "__main__":
    movie_title = "기생충"
    cached_times, non_cached_times = test_caching_performance(movie_title, num_calls=5)


# 서버 실행
# if __name__ == "__main__":
app.launch(server_port=7937, server_name="0.0.0.0", share=True)

INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:__main__:
=== '기생충'에 대한 성능 테스트 시작 (호출 횟수: 5) ===
INFO:__main__:
캐싱 적용 함수 (@lru_cache 사용)
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:__main__:TMDB API 응답 시간: 0.37초
INFO:__main__:호출 1: 0.3778초
INFO:__main__:호출 2: 0.0000초
INFO:__main__:호출 3: 0.0000초
INFO:__main__:호출 4: 0.0000초
INFO:__main__:호출 5: 0.0000초
INFO:__main__:캐싱 적용 평균 실행 시간: 0.0756초
INFO:__main__:
캐싱 미적용 함수
INFO:__main__:호출 1: 0.0000초
INFO:__main__:호출 2: 0.0000초
INFO:__main__:호출 3: 0.0000초
INFO:__main__:호출 4: 0.0000초
INFO:__main__:호출 5: 0.0000초
INFO:__main__:캐싱 미적용 평균 실행 시간: 0.0000초
INFO:__main__:
캐싱으로 인한 속도 향상 배율: 0.00x
INFO:__main__:캐싱 시간 단축량: -0.0756초
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


결과: [{'adult': False, 'backdrop_path': '/hiKmpZMGZsrkA3cdce8a7Dpos1j.jpg', 'genre_ids': [35, 53, 18], 'id': 496243, 'original_language': 'ko', 'original_title': '기생충', 'overview': '전원 백수로 살 길 막막하지만 사이는 좋은 기택 가족. 장남 기우에게 명문대생 친구가 연결시켜 준 고액 과외 자리는 모처럼 싹튼 고정수입의 희망이다. 온 가족의 도움과 기대 속에 박 사장 집으로 향하는 기우. 글로벌 IT기업의 CEO인 박 사장의 저택에 도착하자 젊고 아름다운 사모님 연교와 가정부 문광이 기우를 맞이한다. 큰 문제 없이 박 사장의 딸 다혜의 과외를 시작한 기우. 그러나 이렇게 시작된 두 가족의 만남 뒤로, 걷잡을 수 없는 사건이 기다리고  있는데.....', 'popularity': 18.1871, 'poster_path': '/mSi0gskYpmf1FbXngM37s2HppXh.jpg', 'release_date': '2019-05-30', 'title': '기생충', 'video': False, 'vote_average': 8.499, 'vote_count': 19256}, {'adult': False, 'backdrop_path': '/9VqvhI7mBuqAHit2aWVVaYZ0JCL.jpg', 'genre_ids': [80, 18, 53], 'id': 523077, 'original_language': 'en', 'original_title': 'Running with the Devil', 'overview': '캐나다 벤쿠버에서 대기업을 운영하고 있는 ‘회장’은 남미에서 제조한 마약을 밀반입해 불법으로 유통시켜 짭짤한 부수입을 올리고 있다. 어느 날, 1Kg 당 20g의 마약이 사라지고 있다는 사실을 확인한 그는 오랫동안 자신의 밑에서 마약유통을 책임져 온 ‘요리사’에게 유통경로의 문제점을 파악하라고 지시한다. 이에 ‘

INFO:httpx:HTTP Request: GET http://localhost:7937/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://localhost:7937/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on public URL: https://3706183f0dd04600ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://3706183f0dd04600ea.gradio.live "HTTP/1.1 200 OK"


INFO:__main__:TMDB API 응답 시간: 0.38초


결과: [{'adult': False, 'backdrop_path': '/2PDTWfuBWQKVC7aPAqJK5UCpz08.jpg', 'genre_ids': [28, 878], 'id': 557, 'original_language': 'en', 'original_title': 'Spider-Man', 'overview': '평범하고 내성적인 고등학생 피터 파커는 우연히 방사능에 감염된 거미에 물린다.그 후, 피터는 손에서 거미줄이 튀어 나오고 벽을 기어 오를 수 있는 거미와 같은 능력을 갖게 된다. 다가오는 위험을 본능적으로 감지하는 초감각과 엄청난 파워를 소유하게 된 것이다. 피터는 짝사랑하던 메리 제인의 관심을 끌기 위해 멋진 스포츠카를 구입하는데 초능력을 처음 사용한다. 그러다 사랑하는 벤 아저씨의 죽음을 계기로 그 힘을 악의 세력에 대항하는데 쓰기로 결심한다. 한편 피터의 절친 해리 오스본의 아버지인 노만은 실험 도중 가스에 중독되어 악의 화신 그린 고블린으로 변하는데...', 'popularity': 20.7244, 'poster_path': '/4xOhtzXGt0if74dTmS4qtizFelX.jpg', 'release_date': '2002-05-01', 'title': '스파이더맨', 'video': False, 'vote_average': 7.313, 'vote_count': 19715}, {'adult': False, 'backdrop_path': '/zD5v1E4joAzFvmAEytt7fM3ivyT.jpg', 'genre_ids': [28, 12, 878], 'id': 634649, 'original_language': 'en', 'original_title': 'Spider-Man: No Way Home', 'overview': '미스테리오의 계략으로 세상에 정체가 탄로난 스파이더맨 피터 파커는 하루 아침에 평범한 일상을 잃게 된다. 문제를 해결하기 위해 닥터 스트레인지를 찾아가 도움을 청하지만 뜻하지 않게 멀티버스가 열리면서 각기 다른 

In [ ]:
iface.close()